In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
from tensorboard.plugins.hparams import api as hp

In [3]:
mnist_dataset,mnist_info=tfds.load(name='mnist',with_info=True,as_supervised=True)

In [4]:
mnist_train,mnist_test=mnist_dataset['train'],mnist_dataset['test']

In [6]:
no_of_validation_samples=0.1*mnist_info.splits['train'].num_examples
no_of_validation_samples=tf.cast(no_of_validation_samples,tf.int64)

no_of_test_samples=mnist_info.splits['test'].num_examples
no_of_test_samples=tf.cast(no_of_test_samples,tf.int64)

In [7]:
buffer_size=70000
batch_size=128
max_epoch=20

In [8]:
def scale(image,label):
    image=tf.cast(image,tf.float32)
    image/=255.
    return image,label

In [9]:
scaled_train_and_validation_data=mnist_train.map(scale)
scaled_test_data=mnist_test.map(scale)

In [10]:
suffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(buffer_size)

In [11]:
validation_data=suffled_train_and_validation_data.take(no_of_validation_samples)
train_data=suffled_train_and_validation_data.skip(no_of_validation_samples)

In [12]:
train_data=train_data.batch(batch_size)
validation_data=validation_data.batch(no_of_validation_samples)
test_data=scaled_test_data.batch(no_of_test_samples)

In [15]:
HP_Filter_Size=hp.HParam('filter_size',hp.Discrete([3,5,7]))
HP_optimizer=hp.HParam('optimizer',hp.Discrete(['adam','sgd']))

METRIC_ACCURACY='accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
    hparams=[HP_Filter_Size,HP_optimizer],
    metrics=[hp.Metric(METRIC_ACCURACY,display_name='Accuracy')])

In [24]:
def train_test_model(hparams):
    model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(50,hparams[HP_Filter_Size],activation='relu',input_shape=(28,28,1)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(50,hparams[HP_Filter_Size],activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10)])
    loss_rn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits='softmax')
    model.compile(optimizer=hparams[HP_optimizer],loss=loss_rn,metrics=['accuracy'])
    early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss',mode='auto',min_delta=0,verbose=0,patience=2,restore_best_weights=True)
    model.fit(train_data,epochs=max_epoch,callbacks=[early_stopping],validation_data=validation_data,verbose=2)
    _,accuracy=model.evaluate(test_data)
    return accuracy

In [25]:
def run(log_dir,hparams):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)
        accuracy=train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY,accuracy,step=1)

In [ ]:
session_num=0
for filter_size in HP_Filter_Size.domain.values:
    for optimizer in HP_optimizer.domain.values:
        hparams={
            HP_Filter_Size:filter_size,
            HP_optimizer:optimizer
        }
        run_name='run-%d' % session_num
        print('---Starting trial: %s' % run_name)
        print({h.name:hparams[h] for h in hparams})
        run('logs/hparam_tuning/'+run_name,hparams)
        
        session_num+=1

---Starting trial: run-0
{'filter_size': 3, 'optimizer': 'adam'}
Epoch 1/20
422/422 - 79s - loss: 0.2924 - accuracy: 0.9136 - val_loss: 0.0886 - val_accuracy: 0.9737
Epoch 2/20
422/422 - 78s - loss: 0.0792 - accuracy: 0.9763 - val_loss: 0.0592 - val_accuracy: 0.9827
Epoch 3/20
422/422 - 73s - loss: 0.0569 - accuracy: 0.9825 - val_loss: 0.0443 - val_accuracy: 0.9863
Epoch 4/20
422/422 - 74s - loss: 0.0469 - accuracy: 0.9854 - val_loss: 0.0378 - val_accuracy: 0.9877
Epoch 5/20
422/422 - 78s - loss: 0.0391 - accuracy: 0.9874 - val_loss: 0.0454 - val_accuracy: 0.9873
Epoch 6/20
422/422 - 83s - loss: 0.0345 - accuracy: 0.9894 - val_loss: 0.0230 - val_accuracy: 0.9925
Epoch 7/20
422/422 - 85s - loss: 0.0325 - accuracy: 0.9900 - val_loss: 0.0226 - val_accuracy: 0.9933
Epoch 8/20
422/422 - 87s - loss: 0.0275 - accuracy: 0.9915 - val_loss: 0.0301 - val_accuracy: 0.9913
Epoch 9/20
422/422 - 73s - loss: 0.0242 - accuracy: 0.9925 - val_loss: 0.0191 - val_accuracy: 0.9953
Epoch 10/20
422/422 - 77s 

Epoch 1/20


422/422 - 73s - loss: 0.0144 - accuracy: 0.9953 - val_loss: 0.0192 - val_accuracy: 0.9930
Epoch 2/20
422/422 - 70s - loss: 0.0131 - accuracy: 0.9956 - val_loss: 0.0133 - val_accuracy: 0.9955
Epoch 3/20
422/422 - 69s - loss: 0.0117 - accuracy: 0.9962 - val_loss: 0.0129 - val_accuracy: 0.9958
Epoch 4/20
422/422 - 70s - loss: 0.0099 - accuracy: 0.9969 - val_loss: 0.0068 - val_accuracy: 0.9977
Epoch 5/20
422/422 - 74s - loss: 0.0088 - accuracy: 0.9974 - val_loss: 0.0106 - val_accuracy: 0.9965
Epoch 6/20
422/422 - 75s - loss: 0.0076 - accuracy: 0.9976 - val_loss: 0.0061 - val_accuracy: 0.9977
Epoch 7/20
422/422 - 76s - loss: 0.0079 - accuracy: 0.9975 - val_loss: 0.0064 - val_accuracy: 0.9980
Epoch 8/20
422/422 - 83s - loss: 0.0079 - accuracy: 0.9971 - val_loss: 0.0056 - val_accuracy: 0.9978
Epoch 9/20
422/422 - 89s - loss: 0.0057 - accuracy: 0.9981 - val_loss: 0.0038 - val_accuracy: 0.9990
Epoch 10/20
422/422 - 89s - loss: 0.0057 - accuracy: 0.9982 - val_loss: 0.0044 - val_accuracy: 0.9988


In [21]:
%load_ext tensorboard
%tensorboard --logdir 'logs/hparam_tuning'

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 3184), started 0:00:21 ago. (Use '!kill 3184' to kill it.)

In [22]:
test_loss,test_accuracy=model.evaluate(test_data)

      1/Unknown - 42s 42s/step - loss: 0.0392 - accuracy: 0.9906